In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml


import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
from openff.toolkit.topology import Molecule
import torch


import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


In [2]:
# Use espaloma to find the FF parameters for Benzene
smiles = "c1ccccc1"
benzene = mb.load(smiles, smiles=True)
benzene.visualize().show()

molecule = Molecule.from_smiles(smiles)
molecule_graph = esp.Graph(molecule)

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph)

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
particle_types = []
particle_type_dict = dict()

for i in range(pair_forces.getNumParticles()):
    pair_parms = pair_forces.getParticleParameters(index=i)
    charge = pair_parms[0]
    sigma = pair_parms[1]
    epsilon = pair_parms[2]
    if (charge) not in particle_types:  #this is adding the charge,sigma, and epsilon to the particle_types, but only once. 
        particle_types.append((charge))
    #particle_type_dict[i] = particle_types.index((charge, sigma, epsilon))#adding only the index of the pair_parms, this gives particle types.
    print(particle_types)

#particle_types gives the sigma, charge, and epsilon associated with each unique particle, 
#particle_type_dict only gives the index associated with each unique particle type

[Quantity(value=-0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.13, unit=elementary charge)]
[Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.13, unit=elementary charge)]


In [7]:
print(pair_forces.getNumParticles())

12


In [ ]:
bond_types = []
bond_dict = dict() #creating an empty dictionary to put our parameters in 

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    atom1 = bond_parms[0]
    atom2 = bond_parms[1]
    l0 = bond_parms[2]
    k = bond_parms[3]
    if (atom1, atom2, l0, k) not in bond_types:
        bond_types.append((atom1,atom2,l0,k))
    bond_dict[i] = bond_types
    print(bond_dict)

In [25]:
bond_types = []
bond_type_dict = dict()

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    atom1 = bond_parms[0]
    atom2 = bond_parms[1]
    l0 = bond_parms[2]
    k = bond_parms[3]
    if (atom1,atom2,l0,k) not in bond_types: 
        bond_types.append((atom1,atom2,l0,k))
    bond_type_dict = {"atom1":bond_types[-1][0],"atom2":bond_types[-1][1],"k":bond_types[-1][2],"l0":bond_types[-1][3]}
    print(bond_type_dict)

{'atom1': 0, 'atom2': 1, 'k': Quantity(value=0.14419899337180256, unit=nanometer), 'l0': Quantity(value=130064.45891205409, unit=kilojoule/(nanometer**2*mole))}
{'atom1': 0, 'atom2': 5, 'k': Quantity(value=0.14419899337180256, unit=nanometer), 'l0': Quantity(value=130064.45891205409, unit=kilojoule/(nanometer**2*mole))}
{'atom1': 0, 'atom2': 6, 'k': Quantity(value=0.11234977990290071, unit=nanometer), 'l0': Quantity(value=123800.77912066571, unit=kilojoule/(nanometer**2*mole))}
{'atom1': 1, 'atom2': 2, 'k': Quantity(value=0.14419899337180256, unit=nanometer), 'l0': Quantity(value=130064.45891205409, unit=kilojoule/(nanometer**2*mole))}
{'atom1': 1, 'atom2': 7, 'k': Quantity(value=0.11234977990290071, unit=nanometer), 'l0': Quantity(value=123800.77912066571, unit=kilojoule/(nanometer**2*mole))}
{'atom1': 2, 'atom2': 3, 'k': Quantity(value=0.14419899337180256, unit=nanometer), 'l0': Quantity(value=130064.45891205409, unit=kilojoule/(nanometer**2*mole))}
{'atom1': 2, 'atom2': 8, 'k': Quan

In [33]:
bond_types = []
bond_type_dict = dict()

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    atom1 = bond_parms[0]
    atom2 = bond_parms[1]
    l0 = bond_parms[2]
    k = bond_parms[3]
    bond_type_dict[atom1,atom2]={"k":[k],"l0":[l0]}
print(bond_type_dict)

{(0, 1): {'k': [Quantity(value=130064.45891205409, unit=kilojoule/(nanometer**2*mole))], 'l0': [Quantity(value=0.14419899337180256, unit=nanometer)]}, (0, 5): {'k': [Quantity(value=130064.45891205409, unit=kilojoule/(nanometer**2*mole))], 'l0': [Quantity(value=0.14419899337180256, unit=nanometer)]}, (0, 6): {'k': [Quantity(value=123800.77912066571, unit=kilojoule/(nanometer**2*mole))], 'l0': [Quantity(value=0.11234977990290071, unit=nanometer)]}, (1, 2): {'k': [Quantity(value=130064.45891205409, unit=kilojoule/(nanometer**2*mole))], 'l0': [Quantity(value=0.14419899337180256, unit=nanometer)]}, (1, 7): {'k': [Quantity(value=123800.77912066571, unit=kilojoule/(nanometer**2*mole))], 'l0': [Quantity(value=0.11234977990290071, unit=nanometer)]}, (2, 3): {'k': [Quantity(value=130064.45891205409, unit=kilojoule/(nanometer**2*mole))], 'l0': [Quantity(value=0.14419899337180256, unit=nanometer)]}, (2, 8): {'k': [Quantity(value=123800.77912066571, unit=kilojoule/(nanometer**2*mole))], 'l0': [Quan

In [35]:
type(k)

openmm.unit.quantity.Quantity

In [38]:
k

Quantity(value=123800.77912066571, unit=kilojoule/(nanometer**2*mole))